# LAB-20 班佛定律 Benford's Law

**班佛定律**，也被稱為第一位數法則，是一個關於在許多真實世界數據集中，首位數字出現頻率的觀察。 簡單來說，它指出數字 1 出現為首位數字的機率約為 30%，而數字 9 出現為首位數字的機率則小於 5%。

更詳細的說明：

* 非均勻分佈： 與直覺相反，數字 1 到 9 並非以相同的機率作為數據集中的首位數字。
* 對數分佈： 班佛定律指出，首位數字 d 出現的機率，可以用以下公式計算： P(d) = log10(1 + 1/d)

In [ ]:
import os
import sys
import pandas as pd 
import numpy as np
import matplotlib
import matplotlib.pyplot as plt

In [ ]:
# 使用前次 lab-19.csv 
data_file = "lab-19.csv"
# 讀取 CSV 檔案
df = pd.read_csv(data_file)

In [ ]:
# 設定顯示的最大欄位數
pd.set_option('display.max_columns', None)

In [ ]:
df.head(10)

## 個人捐獻

In [ ]:
# 篩選 
# [收支科目]=個人捐贈收入
# 
df2 = df[df['收支科目'] == '個人捐贈收入'].copy()

# 資料筆數
print(f"資料總筆數：\t{df.shape[0]:10,d}")
print(f"個人捐贈筆數：\t{df2.shape[0]:10,d}")

In [ ]:
# 筆數統計
df2['捐贈方式'].value_counts()

In [ ]:
df2['捐贈方式'].unique()

In [ ]:
df2[df2["捐贈方式"].isnull()]

In [ ]:
# 將空值 置換為 其他
df2['捐贈方式'] = df2['捐贈方式'].fillna("其他")
df2['捐贈方式'].unique()

In [ ]:
# Benford's Law 預期的第一個數字分佈
# 作為繪圖的基礎線
def benford_distribution():
    return [np.log10(1 + 1 / d) for d in range(1, 10)]

In [ ]:
# 統計第一個數字的計數
def calc_first_digit(result, name, amt_val):
    #print(name,amt_val)
    d = int(str(abs(amt_val))[0])                    # 若不考慮負數，則需資料清理
    result[name][d] += 1    

In [ ]:
# 建立一個長度為10的整數陣列，儲存 0-9 數字的累計
cate = df2['捐贈方式'].unique()
benford_data = {}
for c in cate:
    benford_data[c] = np.zeros(10,dtype=int)

In [ ]:
# 逐筆累計
for idx,row in df2.iterrows():
    calc_first_digit(benford_data,row['捐贈方式'], row['收入金額'])

In [ ]:
# 檢視 Ben's Ford Law Data
print(benford_data)

In [ ]:
# 計算分佈
benford_result = {}
for name in cate:
    benford_result[name] = np.zeros(10,dtype=float)
    sub_total = np.sum(benford_data[name][1:10])
    for i in range(1,10):
        benford_result[name][i] = benford_data[name][i]/sub_total

print(benford_result)

In [ ]:
# 班佛定律理論值
expected = benford_distribution()

In [ ]:
# 繪圖中文字型
# 設定中文字型
# 依不同平台 (Windows/Mac) 需設定不同中文字型

if sys.platform == "win32":
    # Windows 
    plt.rcParams['font.family'] = 'SimHei'
elif sys.platform == "darwin":
    plt.rcParams['font.family'] = 'Heiti TC' 
else:
    assert "未知作業系統"

In [ ]:
# 繪圖
digits = range(1, 10)

plt.plot(digits, benford_result['匯款'][1:10], label='匯款',color='green', marker='o', linestyle='-')
plt.plot(digits, benford_result['現金'][1:10], label='現金',color='black', marker='s', linestyle='-')
plt.plot(digits, benford_result['票據'][1:10], label='票據',color='blue', marker='^', linestyle='-')
plt.plot(digits, benford_result['其他'][1:10], label='其他',color='purple', marker='*', linestyle='-')

plt.plot(digits, expected, label="班佛定律", color='red',linestyle='-')

plt.xlabel('數字')
plt.ylabel('分佈')
plt.title("班佛定律分佈 vs 政治獻金(個人捐贈)")
plt.legend()
plt.grid(True)
plt.show()

## 費用支出

In [ ]:
# 篩選 
# [收支]=支出
# 
df3 = df[df["收支"] == "支出"].copy()

# 資料筆數
print(f"資料總筆數：\t{df.shape[0]:10,d}")
print(f"支出筆數：\t{df3.shape[0]:10,d}")

In [ ]:
df3["收支科目"].value_counts()

In [ ]:
# 只分析前四項： 雜支支出, 人事費用支出, 交通旅運支出, 宣傳支出
cate = ['雜支支出','人事費用支出','交通旅運支出','宣傳支出']
df3 = df3[df3['收支科目'].isin(cate)]

In [ ]:
# 支出金額有負值
df3[df3['支出金額'] <= 0].head()

In [ ]:
# 建立一個長度為10的整數陣列，儲存 0-9 數字的累計
display(cate)
benford_data = {}
for name in cate:
    benford_data[name] = np.zeros(10,dtype=int)

In [ ]:
df3["收支科目"].value_counts()

In [ ]:
# 逐筆累計
for idx,row in df3.iterrows():
    calc_first_digit(benford_data, row['收支科目'],row['支出金額'])

In [ ]:
print(benford_data)

In [ ]:
# 計算分佈
benford_result = {}
for name in cate:
    benford_result[name] = np.zeros(10,dtype=float)
    sub_total = np.sum(benford_data[name][1:10])
    for i in range(1,10):
        benford_result[name][i] = benford_data[name][i]/sub_total

print(benford_result)

In [ ]:
# 繪圖
digits = range(1, 10)

plt.plot(digits, benford_result['雜支支出'][1:10], label='雜支',color='green', marker='o', linestyle='-')
plt.plot(digits, benford_result['人事費用支出'][1:10], label='人事',color='black', marker='s', linestyle='-')
plt.plot(digits, benford_result['交通旅運支出'][1:10], label='交通',color='blue', marker='^', linestyle='-')
plt.plot(digits, benford_result['宣傳支出'][1:10], label='宣傳',color='purple', marker='o', linestyle='-')

plt.plot(digits, expected, label="班佛定律", color='red',linestyle='-')

plt.xlabel('數字')
plt.ylabel('分佈')
plt.title("班佛定律分佈 vs 政治獻金(支出)")
plt.legend()
plt.grid(True)
plt.show()

## 總結
* 捐贈收入，同一般紅包，會有喜好區別，如避免 4
* 支出較無上述現象，依上述結果，交通費可能需加重查核